# <center> Squad Selector </center>

Author:  Farhan Kassam

In this notebook, we will solve a FPL version of the mixed-integer linear problem (MILP) or the knapsack problem.

The knapsack problem is a combinatorial optimization problem where you are given a set of items, each with a weight and a value. The program must determine a subset of items to include in the knapsack so that the total weight is less than or equal to a given limit and the total value is as large as possible.

# FPL MILP

In FPL, there is an objective and a set of rules (constraints) that we must adhere to when selecting a team. We will explain the problem in words and mathematical terms.

Objective: Maximize the points earned by a player

$F = \sum_{i=0}^{N}(x_i + y_i) * V_i$

Constraints:
- The cost should be less than 100m (1000 in our dataset since values are not saved as floats)
    - $\sum_{i=0}^{N}(x_i+y_i) * C_i \le 100$
<br></br>
- There are 11 players in the starting lineup
    - $\sum_{i=0}^{N}x_i = 11$
<br></br>
- There are 15 players in the squad
    - $\sum_{i=0}^{N}x_i + y_i = 15$
<br></br>
- There are 2 Goalkeepers in the squad and only 1 in the lineup
    - $\sum_{j \in G}x_j = 1$
    - $\sum_{j \in G}x_j +y_j = 2$
<br></br>  
- There are between 3-5 defenders in the starting lineup and 5 in the squad
    - $3 \le \sum_{j \in D}x_j \le 5$
    - $\sum_{j \in D}x_j + y_j = 5$
<br></br>
- There are between 3-5 midfielders in the starting lineup and 5 in the squad
    - $3 \le \sum_{j \in M}x_j \le 5$
    - $\sum_{j \in M}x_j + y_j = 5$
<br></br> 
- There are between 1-3 forwards in the starting lineup and 3 in the squad
    - $1 \le \sum_{j \in F}x_j \le 3$
    - $\sum_{j \in F}x_j + y_j \le 3$
<br></br>
- There cannot be more than 3 players from the same team in the squad
    - $\sum_{j \in T_k}x_j + y_j \le 3$

Let's start by selecting a team based on the above constraints for the season so far via the aggreated data and a python library specifically for MILP called pulp.

In [1]:
import pandas as pd
import numpy as np
import pulp

In [2]:
agg = pd.read_csv('../data/aggregated.csv', index_col=0)
agg.head()

,name,team,position,total_points,minutes,goals_scored,assists,clean_sheets,saves,penalties_saved,penalties_missed,goals_conceded,own_goals,yellow_cards,red_cards,value,PV-ratio
0,Erling Haaland,Man City,FWD,189,1950,27,4,7,0,0,0,23,0,4,0,122,1.549180
1,Harry Kane,Spurs,FWD,157,2236,17,6,9,0,0,1,36,0,4,0,118,1.330508
2,Marcus Rashford,Man Utd,MID,156,1999,15,4,9,0,0,0,25,0,3,0,73,2.136986
3,Kieran Trippier,Newcastle,DEF,154,2107,1,5,15,0,0,0,13,0,6,0,61,2.524590
4,Martin Ødegaard,Arsenal,MID,144,1954,9,8,8,0,0,0,24,0,3,0,70,2.057143


Below we will create helper variables which will help us set and adhere to the models constraints.

In [3]:
# Creating helper variables
POS = agg['position'].unique()
CLUBS = agg['team'].unique()
budget = 1000
pos_available = {'GK': 2, 'DEF': 5, 'MID': 5, 'FWD': 3}

positions = np.array(agg.position)
costs = np.array(agg.value)
points = np.array(agg.total_points)
teams = np.array(agg.team)

In [4]:
# initializing the model
model = pulp.LpProblem("FPL-Optimization", pulp.LpMaximize)
# decision types
# the format function inserts i into empty placeholder {} to create a list of possible inclusions for the model

lineup = [pulp.LpVariable("x_{}".format(i), lowBound = 0, upBound = 1, cat = 'Integer') for i in range(len(agg))]
subs = [pulp.LpVariable("y_{}".format(i), lowBound = 0, upBound = 1, cat = 'Integer') for i in range(len(agg))]

# defining model objective

model += pulp.lpSum((lineup[i] + subs[i]*0.1) * points[i] for i in range(len(agg))), "Objective"

# defining constraints

# Budget constraint
model += pulp.lpSum((lineup[i] + subs[i]) * costs[i] for i in range(len(agg))) <= budget

# Starting Goalkeeper constraint
model += pulp.lpSum(lineup[i] for i in range(len(agg)) if positions[i] == 'GK') == 1

# Starting Defender constraint
model += pulp.lpSum(lineup[i] for i in range(len(agg)) if positions[i] == 'DEF') >= 3
model += pulp.lpSum(lineup[i] for i in range(len(agg)) if positions[i] == 'DEF') <= 5

# Starting Midfielder constraint
model += pulp.lpSum(lineup[i] for i in range(len(agg)) if positions[i] == 'MID') >= 3
model += pulp.lpSum(lineup[i] for i in range(len(agg)) if positions[i] == 'MID') <= 5

# Starting Forward constraint
model += pulp.lpSum(lineup[i] for i in range(len(agg)) if positions[i] == 'FWD') >= 1
model += pulp.lpSum(lineup[i] for i in range(len(agg)) if positions[i] == 'FWD') <= 3

# Team position constraints
for pos in POS:
    model += pulp.lpSum(lineup[i] + subs[i] for i in range(len(agg)) if positions[i] == pos) == pos_available[pos]

# Club constraint for team
for club in CLUBS:
    model += pulp.lpSum(lineup[i] + subs[i] for i in range(len(agg)) if teams[i] == club) <= 3

# Lineup size constraint

model += pulp.lpSum(lineup[i] for i in range(len(agg))) == 11

# total team size constraint

model += pulp.lpSum(lineup[i] + subs[i] for i in range(len(agg))) == 15

for i in range(len(agg)):
    model += (lineup[i] + subs[i]) <= 1  # subs must not be on team
    
model.solve()

1

The above says the model was successful in determining a solution. The following code blocks will extract the players and create a dataframe of the squad selected by the MILP model.

In [5]:
squad_array = []
for i in range(len(lineup)):
    if lineup[i].value() != 0:
        squad_array.append([agg.name[i], agg.team[i], agg.position[i], agg.total_points[i], agg.value[i]])
    if subs[i].value() != 0:
        squad_array.append([agg.name[i], agg.team[i], agg.position[i], agg.total_points[i], agg.value[i]])
    
squad_df = pd.DataFrame(data=squad_array,columns=['name', 'team', 'position', 'points','value'])
    
print(f"Total Score = {model.objective.value()}")
print(f"Squad Cost = {squad_df.value.sum()}")
squad_df

Total Score = 1563.3000000000002
Squad Cost = 1000


,name,team,position,points,value
0,Erling Haaland,Man City,FWD,189,122
1,Harry Kane,Spurs,FWD,157,118
2,Marcus Rashford,Man Utd,MID,156,73
3,Kieran Trippier,Newcastle,DEF,154,61
4,Martin Ødegaard,Arsenal,MID,144,70
5,Bukayo Saka,Arsenal,MID,138,84
6,Ivan Toney,Brentford,FWD,134,77
7,Miguel Almirón Rejala,Newcastle,MID,128,56
8,David Raya Martin,Brentford,GK,116,47
9,Fabian Schär,Newcastle,DEF,108,52


The above is the best overall squad in the season so far where the first 11 entries are the starting XI and the remaining 4 entries are the substitutes. The total score of this squad so far is 1563 and the total cost is 1000 using the entire budget.

# Generalizing The Problem

Since the model worked, we can now generalize it to a function where we can select a squad based on the `xP` column on a week by week basis. We can create another function to select a squad based on the ground truth (`total_points`) column then comparing the two to determine if the expected points squad did as well as the truly highest scoring squad in a particular week. Lastly, we can create another function to select a squad based on our model's predictions `mypred` and compare the three selected squads.

Function to select squad based on `xP` column.

In [6]:
def xP_squad(data, budget):
    '''This function returns a 15-man squad in a dataframe where the first 11 are the starting lineup and the last 4
    are subs. The squad is returned based on mixed-integer linear programming with xP column as the objective.'''
    
    assert isinstance(data, pd.DataFrame), "Data Must Be Pandas DataFrame"
    assert isinstance(budget, int), "Budget Must Be Integer"
    assert set(['position', 'team', 'value', 'xP']).issubset(data.columns), "Must Have Required Columns: position, team, value, xP"
    
    # Helper Variables
    POS = data['position'].unique()
    CLUBS = data['team'].unique()
    budget = budget
    pos_available = {'GK': 2, 'DEF': 5, 'MID': 5, 'FWD': 3}

    positions = np.array(data.position)
    costs = np.array(data.value)
    points = np.array(data.xP)
    teams = np.array(data.team)
    
    # initializing the model
    model = pulp.LpProblem("FPL-Optimization", pulp.LpMaximize)
    # decision types
    # the format function inserts i into empty placeholder {} to create a list of possible inclusions for the model

    lineup = [pulp.LpVariable("x_{}".format(i), lowBound = 0, upBound = 1, cat = 'Integer') for i in range(len(data))]
    subs = [pulp.LpVariable("y_{}".format(i), lowBound = 0, upBound = 1, cat = 'Integer') for i in range(len(data))]

    # defining model objective

    model += pulp.lpSum((lineup[i] + subs[i]*0.1) * points[i] for i in range(len(data))), "Objective"

    # defining constraints

    # Budget constraint
    model += pulp.lpSum((lineup[i] + subs[i]) * costs[i] for i in range(len(data))) <= budget

    # Starting Goalkeeper constraint
    model += pulp.lpSum(lineup[i] for i in range(len(data)) if positions[i] == 'GK') == 1

    # Starting Defender constraint
    model += pulp.lpSum(lineup[i] for i in range(len(data)) if positions[i] == 'DEF') >= 3
    model += pulp.lpSum(lineup[i] for i in range(len(data)) if positions[i] == 'DEF') <= 5

    # Starting Midfielder constraint
    model += pulp.lpSum(lineup[i] for i in range(len(data)) if positions[i] == 'MID') >= 3
    model += pulp.lpSum(lineup[i] for i in range(len(data)) if positions[i] == 'MID') <= 5

    # Starting Forward constraint
    model += pulp.lpSum(lineup[i] for i in range(len(data)) if positions[i] == 'FWD') >= 1
    model += pulp.lpSum(lineup[i] for i in range(len(data)) if positions[i] == 'FWD') <= 3

    # Team position constraints
    for pos in POS:
        model += pulp.lpSum(lineup[i] + subs[i] for i in range(len(data)) if positions[i] == pos) == pos_available[pos]

    # Club constraint for team
    for club in CLUBS:
        model += pulp.lpSum(lineup[i] + subs[i] for i in range(len(data)) if teams[i] == club) <= 3

    # Lineup size constraint

    model += pulp.lpSum(lineup[i] for i in range(len(data))) == 11

    # total team size constraint

    model += pulp.lpSum(lineup[i] + subs[i] for i in range(len(data))) == 15

    for i in range(len(data)):
        model += (lineup[i] + subs[i]) <= 1  # subs must not be on team

    model.solve()
    
    squad_array = []
    for i in range(len(lineup)):
        if lineup[i].value() != 0:
            squad_array.append([data.name[i], data.team[i], data.position[i],  data.GW[i], data.xP[i], data.value[i]])
        if subs[i].value() != 0:
            squad_array.append([data.name[i], data.team[i], data.position[i],  data.GW[i], data.xP[i], data.value[i]])
    
    squad_df = pd.DataFrame(data=squad_array,columns=['name', 'team', 'position', 'GW', 'xP','value'])
    
    print(f"Total Score = {model.objective.value()}\nSquad Value = {squad_df.value.sum()}\n\n")
    return squad_df

Function to select my model's predicted squad.

In [7]:
def mypred_squad(data, budget):
    '''This function returns a 15-man squad in a dataframe where the first 11 are the starting lineup and the last 4
    are subs. The squad is returned based on mixed-integer linear programming with mypred column as the objective.'''
    
    assert isinstance(data, pd.DataFrame), "Data Must Be Pandas DataFrame"
    assert isinstance(budget, int), "Budget Must Be Integer"
    assert set(['position', 'team', 'value', 'mypred']).issubset(data.columns), "Must Have Required Columns: position, team, value, mypred"
    
    # Helper Variables
    POS = data['position'].unique()
    CLUBS = data['team'].unique()
    budget = budget
    pos_available = {'GK': 2, 'DEF': 5, 'MID': 5, 'FWD': 3}

    positions = np.array(data.position)
    costs = np.array(data.value)
    points = np.array(data.mypred)
    teams = np.array(data.team)
    
    # initializing the model
    model = pulp.LpProblem("FPL-Optimization", pulp.LpMaximize)
    # decision types
    # the format function inserts i into empty placeholder {} to create a list of possible inclusions for the model

    lineup = [pulp.LpVariable("x_{}".format(i), lowBound = 0, upBound = 1, cat = 'Integer') for i in range(len(data))]
    subs = [pulp.LpVariable("y_{}".format(i), lowBound = 0, upBound = 1, cat = 'Integer') for i in range(len(data))]

    # defining model objective

    model += pulp.lpSum((lineup[i] + subs[i]*0.1) * points[i] for i in range(len(data))), "Objective"

    # defining constraints

    # Budget constraint
    model += pulp.lpSum((lineup[i] + subs[i]) * costs[i] for i in range(len(data))) <= budget

    # Starting Goalkeeper constraint
    model += pulp.lpSum(lineup[i] for i in range(len(data)) if positions[i] == 'GK') == 1

    # Starting Defender constraint
    model += pulp.lpSum(lineup[i] for i in range(len(data)) if positions[i] == 'DEF') >= 3
    model += pulp.lpSum(lineup[i] for i in range(len(data)) if positions[i] == 'DEF') <= 5

    # Starting Midfielder constraint
    model += pulp.lpSum(lineup[i] for i in range(len(data)) if positions[i] == 'MID') >= 3
    model += pulp.lpSum(lineup[i] for i in range(len(data)) if positions[i] == 'MID') <= 5

    # Starting Forward constraint
    model += pulp.lpSum(lineup[i] for i in range(len(data)) if positions[i] == 'FWD') >= 1
    model += pulp.lpSum(lineup[i] for i in range(len(data)) if positions[i] == 'FWD') <= 3

    # Team position constraints
    for pos in POS:
        model += pulp.lpSum(lineup[i] + subs[i] for i in range(len(data)) if positions[i] == pos) == pos_available[pos]

    # Club constraint for team
    for club in CLUBS:
        model += pulp.lpSum(lineup[i] + subs[i] for i in range(len(data)) if teams[i] == club) <= 3

    # Lineup size constraint

    model += pulp.lpSum(lineup[i] for i in range(len(data))) == 11

    # total team size constraint

    model += pulp.lpSum(lineup[i] + subs[i] for i in range(len(data))) == 15

    for i in range(len(data)):
        model += (lineup[i] + subs[i]) <= 1  # subs must not be on team

    model.solve()
    
    squad_array = []
    for i in range(len(lineup)):
        if lineup[i].value() != 0:
            squad_array.append([data.name[i], data.team[i], data.position[i],  data.GW[i], data.mypred[i], data.value[i]])
        if subs[i].value() != 0:
            squad_array.append([data.name[i], data.team[i], data.position[i],  data.GW[i], data.mypred[i], data.value[i]])
    
    squad_df = pd.DataFrame(data=squad_array,columns=['name', 'team', 'position', 'GW', 'mypred','value'])
    
    print(f"Total Score = {model.objective.value()}\nSquad Value = {squad_df.value.sum()}\n\n")
    return squad_df

Function to return actual results squad.

In [8]:
def true_squad(data, budget):
    '''This function returns a 15-man squad in a dataframe where the first 11 are the starting lineup and the last 4
    are subs. The squad is returned based on mixed-integer linear programming with actual_points column as the objective.'''
    
    assert isinstance(data, pd.DataFrame), "Data Must Be Pandas DataFrame"
    assert isinstance(budget, int), "Budget Must Be Integer"
    assert set(['position', 'team', 'value', 'actual_points']).issubset(data.columns), "Must Have Required Columns: position, team, value, actual_points"
    
    # Helper Variables
    POS = data['position'].unique()
    CLUBS = data['team'].unique()
    budget = budget
    pos_available = {'GK': 2, 'DEF': 5, 'MID': 5, 'FWD': 3}

    positions = np.array(data.position)
    costs = np.array(data.value)
    points = np.array(data.actual_points)
    teams = np.array(data.team)
    
    # initializing the model
    model = pulp.LpProblem("FPL-Optimization", pulp.LpMaximize)
    # decision types
    # the format function inserts i into empty placeholder {} to create a list of possible inclusions for the model

    lineup = [pulp.LpVariable("x_{}".format(i), lowBound = 0, upBound = 1, cat = 'Integer') for i in range(len(data))]
    subs = [pulp.LpVariable("y_{}".format(i), lowBound = 0, upBound = 1, cat = 'Integer') for i in range(len(data))]

    # defining model objective

    model += pulp.lpSum((lineup[i] + subs[i]*0.1) * points[i] for i in range(len(data))), "Objective"

    # defining constraints

    # Budget constraint
    model += pulp.lpSum((lineup[i] + subs[i]) * costs[i] for i in range(len(data))) <= budget

    # Starting Goalkeeper constraint
    model += pulp.lpSum(lineup[i] for i in range(len(data)) if positions[i] == 'GK') == 1

    # Starting Defender constraint
    model += pulp.lpSum(lineup[i] for i in range(len(data)) if positions[i] == 'DEF') >= 3
    model += pulp.lpSum(lineup[i] for i in range(len(data)) if positions[i] == 'DEF') <= 5

    # Starting Midfielder constraint
    model += pulp.lpSum(lineup[i] for i in range(len(data)) if positions[i] == 'MID') >= 3
    model += pulp.lpSum(lineup[i] for i in range(len(data)) if positions[i] == 'MID') <= 5

    # Starting Forward constraint
    model += pulp.lpSum(lineup[i] for i in range(len(data)) if positions[i] == 'FWD') >= 1
    model += pulp.lpSum(lineup[i] for i in range(len(data)) if positions[i] == 'FWD') <= 3

    # Team position constraints
    for pos in POS:
        model += pulp.lpSum(lineup[i] + subs[i] for i in range(len(data)) if positions[i] == pos) == pos_available[pos]

    # Club constraint for team
    for club in CLUBS:
        model += pulp.lpSum(lineup[i] + subs[i] for i in range(len(data)) if teams[i] == club) <= 3

    # Lineup size constraint

    model += pulp.lpSum(lineup[i] for i in range(len(data))) == 11

    # total team size constraint

    model += pulp.lpSum(lineup[i] + subs[i] for i in range(len(data))) == 15

    for i in range(len(data)):
        model += (lineup[i] + subs[i]) <= 1  # subs must not be on team

    model.solve()
    
    squad_array = []
    for i in range(len(lineup)):
        if lineup[i].value() != 0:
            squad_array.append([data.name[i], data.team[i], data.position[i],  data.GW[i], data.actual_points[i], data.value[i]])
        if subs[i].value() != 0:
            squad_array.append([data.name[i], data.team[i], data.position[i],  data.GW[i], data.actual_points[i], data.value[i]])
    
    squad_df = pd.DataFrame(data=squad_array,columns=['name', 'team', 'position', 'GW', 'actual_points','value'])
    
    print(f"Total Score = {model.objective.value()}\nSquad Value = {squad_df.value.sum()}\n\n")
    return squad_df

Function that returns all three squads from the predictions dataframe in the Modeling notebook.

In [9]:
def squads(data, budget):
    '''Returns 3 different squads:
    1. FPL Predicted squad, 
    2. RandomForest Predicted squad
    3. True Results Squad'''
    
    xP_team = xP_squad(data, budget)
    mypred_team = mypred_squad(data, budget)
    true_team = true_squad(data, budget)
    return xP_team, mypred_team, true_team

# Prediction Squads vs Actual Squad

Now that we have defined all the necessary functions, we will compare the prediction squads to the true squad by the percent match of players selected and accuracy of the predicted points column compared to the actual_points.

In [17]:
# reading in the predictions data
pred_df = pd.read_csv('../data/pred_df.csv', index_col = 0)
pred_df

,name,position,team,GW,value,xP,mypred,actual_points
0,Fabian Schär,DEF,Newcastle,21,51,6.0,7.327581,6
1,Brennan Johnson,FWD,Nott'm Forest,21,56,4.5,5.464833,5
2,Cheick Doucouré,MID,Crystal Palace,21,50,2.0,1.995472,2
3,Alisson Ramses Becker,GK,Liverpool,21,55,4.1,7.012801,9
4,Che Adams,FWD,Southampton,21,63,4.0,1.996585,2
...,...,...,...,...,...,...,...,...
1252,Ethan Pinnock,DEF,Brentford,24,44,3.3,1.991160,2
1253,Vicente Guaita,GK,Crystal Palace,24,44,3.0,2.537948,3
1254,Nick Pope,GK,Newcastle,24,55,0.7,-2.100000,-3
1255,Oliver Skipp,MID,Spurs,24,43,0.7,1.995472,2


In [20]:
# Determining the number of correct results between xP and actual_points, and between mypred and actual_points
pred_df['Match_xP_actual'] = pred_df['xP'].eq(pred_df['actual_points'])
pred_df['Match_mypred_actual'] = pred_df['mypred'].round().eq(pred_df['actual_points'])

# viewing the dataframe again
pred_df

In [43]:
# Calculating the number of correct predictions from xP and mypred
print(f"xP Correct Percentage: {len(pred_df.loc[pred_df['Match_xP_actual']==True]) / len(pred_df)*100}")
print(f"mypred Correct Percentage: {len(pred_df.loc[pred_df['Match_mypred_actual']==True]) / len(pred_df)*100}")

xP Correct Percentage: 6.443914081145586
mypred Correct Percentage: 84.96420047732697


The results show that from GW 21-24, my model correctly predicted 85% whereas the FPL predictor only got 6.44% correct. An important thing to keep in mind is that the model I created used features that can only be used after a game is completed whereas the FPL xP must be from a truly predictive model explaining their poor performance.

Let's check what the selected teams results would have been!

In [11]:
# Extracting predicted and true squads for gameweek 24

xP_team, mypred_team, true_team = squads(pred_df.loc[pred_df['GW']==24].reset_index(), 1000)

Total Score = 90.32000000000001
Squad Value = 974


Total Score = 119.95976273161801
Squad Value = 925


Total Score = 131.1
Squad Value = 948




The predicted score in: 
- FPL predicted model was 90.32 points with a team value of 974
- My predicted model was 119.95 points with a team value of 925

In contrast, the true value was:
- Total score of 131 points and a team value of 948

Let's see if there are any similar player names between the true selection and the predicted squads.

In [38]:
# seeing if there are similar players in FPL xP and my predicted squad
set(xP_team['name']).intersection(mypred_team['name'])

{'Bernd Leno',
 'Bruno Borges Fernandes',
 'Emerson Leite de Souza Junior',
 'Marcus Rashford',
 'Ollie Watkins',
 'Seamus Coleman'}

In [40]:
# seeing if there are similar players in FPL xP and true squad
set(true_team['name']).intersection(xP_team['name'])

{'Bernd Leno',
 'Bruno Borges Fernandes',
 'Emerson Leite de Souza Junior',
 'James Tarkowski',
 'Marcus Rashford',
 'Ollie Watkins',
 'Seamus Coleman'}

In [41]:
# seeing if there are similar players in my predicted and true squad
set(true_team['name']).intersection(mypred_team['name'])

{'Ben Davies',
 'Bernardo Veiga de Carvalho e Silva',
 'Bernd Leno',
 'Bruno Borges Fernandes',
 'Chris Wood',
 'Emerson Leite de Souza Junior',
 'James Ward-Prowse',
 'Marcus Rashford',
 'Norberto Murara Neto',
 'Ollie Watkins',
 'Seamus Coleman',
 'Trent Alexander-Arnold'}

The percentage match between:
- FPL xP team and my team: 40%
- FPL xP team and true squad: 46%
- My team and true squad: 80%

Below we will view the true squad and then summarize the results.

In [14]:
true_team

,name,team,position,GW,actual_points,value
0,Chris Wood,Nott'm Forest,FWD,24,8,56
1,Bruno Borges Fernandes,Man Utd,MID,24,12,99
2,Bernardo Veiga de Carvalho e Silva,Man City,MID,24,10,68
3,Marcus Rashford,Man Utd,MID,24,15,73
4,Trent Alexander-Arnold,Liverpool,DEF,24,12,73
5,Bernd Leno,Fulham,GK,24,11,45
6,Emerson Leite de Souza Junior,Spurs,DEF,24,15,49
7,Ollie Watkins,Aston Villa,FWD,24,8,71
8,Norberto Murara Neto,Bournemouth,GK,24,10,45
9,James Tarkowski,Everton,DEF,24,8,43


## FINISH THE SUMMARY TMRW
It seems that my predicitive model was better at predicting 